In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("API_KEY"),  # dummy key
    base_url=os.getenv("BASE_URL"),
)

model_name = os.getenv("MODEL_NAME")

In [5]:
response = client.chat.completions.create(
    model= model_name,
    messages=[
        {"role": "user", "content": "What's the capital of Japan?"}
    ],
    temperature = 0.0, 
)

print(response.choices[0].message.content)

The capital of Japan is Tokyo ().


## Get LLM Response

In [124]:
def  get_llm_response(client,model_name, messages, temperature = 0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message['role'], "content": message['content']})
    
    response = client.chat.completions.create(
        model= model_name,
        messages= input_messages,
        temperature = temperature, 
    ).choices[0].message.content
    
    return response


In [125]:
messages = [{"role": "system", "content": "What's the capital of Japan?"}]
response = get_llm_response(client, model_name, messages)


In [126]:

print(response )

The capital of Japan is Tokyo.


## Prompt Engineering


### Structured Output

In [19]:
system_prompt = """
    You are a helpful assistant that answer question about capital of countries.
    
    Your output must be a valid JSON array using double quotes only. Do not include any text outside of the JSON.

    [
    {
        "Country": the country that you will get the capital of
        "Capital": capital of the country stated
    }
    ]
"""

messages= [{'role':'system', 'content':system_prompt}]
messages.append({"role": "system", "content": "What's the capital of Japan?"})
response = get_llm_response(client, model_name, messages)
print(response )

[
{
"Country": "Japan",
"Capital": "Tokyo"
}
]


In [17]:
type(response)

str

In [20]:
import json


json_response = json.loads(response)
json_response

[{'Country': 'Japan', 'Capital': 'Tokyo'}]

In [22]:
type(json_response[0]),json_response[0]['Capital']

(dict, 'Tokyo')

### Input Structuring

In [23]:
user_input = """
Get me the capitals of the following countries:
```
1. Japan 
2. Thailand
3. Singapore
```
"""
messages= [{'role':'system', 'content':system_prompt}]
messages.append({"role": "system", "content": user_input})
response = get_llm_response(client, model_name, messages)
print(response )

[
{
"Country": "Japan",
"Capital": "Tokyo"
},
{
"Country": "Thailand",
"Capital": "Bangkok"
},
{
"Country": "Singapore",
"Capital": "Singapore"
}
]


In [24]:

json_response = json.loads(response)
json_response

[{'Country': 'Japan', 'Capital': 'Tokyo'},
 {'Country': 'Thailand', 'Capital': 'Bangkok'},
 {'Country': 'Singapore', 'Capital': 'Singapore'}]

### Chain of Thought

#### Give the model time to think

In [26]:
user_prompt = """ 
Callculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the example below. 
You are not allowed to write anything other than the json object:
{
    result: The final number result from calculatinig the equation above
}

"""


messages = [{"role": "system", "content": user_prompt}]
response = get_llm_response(client, model_name, messages)
print(response )

{
    "result": 4
}


In [32]:
equation = "259/2*8654+91072*33-19271"

user_prompt = f""" 
Calculate the result of this equation: {equation}

Your output should be in a structured json format exactly like the example below. 
You are not allowed to write anything other than the json object:
{{
    "result": The final number result from calculating the equation above
}}

"""


messages = [{"role": "system", "content": user_prompt}]
response = get_llm_response(client, model_name, messages)
print(response )

{
    "result": 434,841
}


In [33]:
259/2*8654+91072*33-19271

4106798.0

In [34]:
equation = "259/2*8654+91072*33-19271"

user_prompt = f""" 
Calculate the result of this equation: {equation}

Your output should be in a structured json format exactly like the example below. 
You are not allowed to write anything other than the json object:
{{
    "steps": This is where you solve the equation bit by bit following the BEDMAS order of operation. 
                You need to show your works and calculate each step leading to final results.
                Feel free to write in free text.
    "result": The final number result from calculating the equation above
}}

"""


messages = [{"role": "system", "content": user_prompt}]
response = get_llm_response(client, model_name, messages)
print(response )

{
    "steps": "First, we'll follow the BEDMAS order of operations: 1. Divide 259 by 2: 259/2 = 129.5\n2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121433.0\n3. Multiply 91072 by 33: 91072 * 33 = 3004416.0\n4. Add the results of steps 1 and 3: 1121433.0 + 3004416.0 = 4125859.0\n5. Subtract 19271 from the result: 4125859.0 - 19271 = 4104188.0",
    "result": 4104188.0


## RAG - Retrieval Augmented Generation

In [36]:
user_prompt ="""
What's new in iphone 16?
"""

messages = [{'role':'user', 'content': user_prompt}]
response = get_llm_response(client, model_name, messages)
print(response)

There is no such thing as an "iPhone 16". Apple has only released up to iPhone 14 series, with the latest being iPhone 14 Pro and iPhone 14 Pro Max. There has been no official announcement or release of an iPhone 16.

However, if you're looking for information on the latest features and updates in the iPhone 14 series, I can provide you with some details:

* Improved cameras: The iPhone 14 Pro and Pro Max have a new 48MP main camera, a 12MP front camera, and a telephoto lens with 3x optical zoom.
* A15 Bionic chip: The iPhone 14 series features Apple's latest A15 Bionic chip, which provides faster performance and improved power efficiency.
* Longer battery life: The iPhone 14 Pro and Pro Max have longer battery life, with up to 12 hours of internet use on a single charge.
* Fast charging: The iPhone 14 series supports fast charging, allowing you to charge your device up to 50% in just 30 minutes.
* New colors: The iPhone 14 series is available in new colors, including Starlight, Midnig

In [64]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [38]:
user_prompt =f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user', 'content': user_prompt}]
response = get_llm_response(client, model_name, messages)
print(response)

According to the text, some of the new features and changes in the iPhone 16 include:

1. No Apple stickers included in the box (due to environmental reasons)
2. Vertical camera layout, similar to iPhone 12
3. Updated 48-megapixel Fusion camera with improved features such as super-high-resolution photos (24MP and 48MP)
4. New ultrawide camera with a wider aperture and automatic focus
5. Improved thermal design for up to 30% higher sustained performance for gaming
6. Support for WiFi 7 in all models


### Embedding

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en-v1.5")

sentences = ["I love machine learning", "AI is amazing"]
embeddings = model.encode(sentences)

print(embeddings.shape)  # e.g. (2, 384)


c:\Users\crab\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\crab\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\crab\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admi

(2, 384)


### Automatically Extract context data from Database

In [65]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [66]:
data = [iphone_16, samsung_s23]

In [68]:
user_prompt = " What's new in iphone 16?"

In [96]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_client = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device":"cuda"},
    encode_kwargs={"normalize_embeddings": True}
    
)

def get_embedding(embedding_client, text_input): 
    embeddings = embedding_client.embed_query(text_input) 
    
    return embeddings
# embedding = embedding_client.embed_documents(user_prompt)

In [97]:
output = get_embedding(embedding_client, user_prompt)

In [101]:
user_prompt_embedding = get_embedding(embedding_client, user_prompt)

In [102]:
len(user_prompt_embedding)

384

In [107]:
data_embedding = [get_embedding(embedding_client, text)for text in data]

In [108]:
len(data_embedding)

2

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

In [111]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embedding)

In [112]:
data_similarity

array([[0.73789107, 0.5264605 ]])

In [116]:
closet_entry_index = data_similarity.argmax()
closet_entry_index

0

In [118]:
data[closet_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [121]:
user_prompt_with_data = f"""

{data[closet_entry_index]}
\n\n
{user_prompt}
"""

In [123]:
print(user_prompt_with_data)




The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learni

In [127]:
messages = [{'role':'user', 'content':user_prompt_with_data}]
response = get_llm_response(client, model_name, messages)

In [128]:
print(response)

According to the text, the following are new features and updates in the iPhone 16:

1. Larger displays:
	* Base model: 6.1-inch
	* iPhone 16 Plus: 6.7-inch
	* iPhone 16 Pro and Pro Max: 6.3 and 6.9 inches respectively
2. Thinner bezels and Ceramic Shield for durability
3. New A18 chip (A18 Pro for Pro models) with:
	* Enhanced neural engine capabilities
	* Faster GPU for gaming and machine learning tasks
4. Upgraded camera systems:
	* Base iPhone 16: Dual-camera setup with a 48MP main sensor
	* Pro models: 48MP Ultra Wide and 5x telephoto camera, with "Camera Control" button
5. Advanced audio features:
	* "Audio Mix" for refined audio capture during video recording
6. Extended battery life, especially in the iPhone 16 Pro Max
7. Switch to USB-C for faster charging and data transfer
8. Support for up to 2x faster video encoding (in Pro models)
9. Starting prices remain consistent with previous generations:
	* iPhone 16: $799
	* Pro models: $999
